In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from BGWOPSO import BGWOPSO
from GWO import GWO
import functools
import pickle

In [2]:
data = pd.read_csv("./data/employee.csv")
attrition = data["Attrition"]
data = data.drop("Attrition",axis=1)
data["Attrition"] = attrition
# data.to_csv("./data/employee_class_right.csv", index=False)
data

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,41,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,...,80,0,8,0,1,6,4,0,5,Yes
1,49,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,...,80,1,10,3,3,10,7,1,7,No
2,37,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,...,80,0,7,3,3,0,0,0,0,Yes
3,33,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,...,80,0,8,3,3,8,7,3,0,No
4,27,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,...,80,1,6,3,3,2,2,2,2,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,3,...,80,1,17,3,3,5,2,0,3,No
1466,39,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,4,...,80,1,9,5,3,7,7,1,7,No
1467,27,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,2,...,80,1,6,0,3,6,2,0,3,No
1468,49,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,4,...,80,0,17,3,2,9,6,0,8,No


In [7]:
data["Attrition"].value_counts()

No     1233
Yes     237
Name: Attrition, dtype: int64

In [9]:
237/1470

0.16122448979591836

In [ ]:
data.nunique()

In [ ]:
data.describe()


In [ ]:
data.info()

In [25]:
objectList = data.select_dtypes(include = "object").columns.to_list()
nonObjectList = data.select_dtypes(exclude="object").columns.to_list()

In [26]:
for x in ["Education", 'EnvironmentSatisfaction', "JobInvolvement", "JobLevel", "JobSatisfaction", "PerformanceRating", "RelationshipSatisfaction", "StockOptionLevel", "WorkLifeBalance"]:
    nonObjectList.remove(x)
    objectList.append(x)

In [ ]:
print(nonObjectList)
print(objectList)

In [32]:

le = LabelEncoder()
data_encoded = data.copy()

for object in objectList:
    le.fit(data[object])
    data_encoded[object] = le.transform(data[object])
    pickle.dump(le, open('./model/encode/{}.pkl'.format(object), 'wb'))

# data.to_csv("./data/data_encoded.csv", index=False)
# pickle.dump(scaler, open('./model/minmaxscaler.pkl', 'wb'))
data_encoded

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,41,2,1102,2,1,1,1,1,1,1,...,80,0,8,0,0,6,4,0,5,1
1,49,1,279,1,8,0,1,1,2,2,...,80,1,10,3,2,10,7,1,7,0
2,37,2,1373,1,2,1,4,1,4,3,...,80,0,7,3,2,0,0,0,0,1
3,33,1,1392,1,3,3,1,1,5,3,...,80,0,8,3,2,8,7,3,0,0
4,27,2,591,1,2,0,3,1,7,0,...,80,1,6,3,2,2,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,1,884,1,23,1,3,1,2061,2,...,80,1,17,3,2,5,2,0,3,0
1466,39,2,613,1,6,0,3,1,2062,3,...,80,1,9,5,2,7,7,1,7,0
1467,27,2,155,1,4,2,1,1,2064,1,...,80,1,6,0,2,6,2,0,3,0
1468,49,1,1023,2,2,2,3,1,2065,3,...,80,0,17,3,1,9,6,0,8,0


In [103]:
np.sort(data_encoded['MonthlyIncome'].unique())


array([ 1009,  1051,  1052, ..., 19943, 19973, 19999], dtype=int64)

In [92]:
scaler = MinMaxScaler()
data_normalized = data.copy()

for column in data_encoded.columns:
    scaler.fit(data_encoded[column].values.reshape(-1,1))
    data_normalized[column] = scaler.transform(data_encoded[column].values.reshape(-1,1))
    pickle.dump(scaler, open('./model/scale/{}.pkl'.format(column), 'wb'))
data_normalized.to_csv("./data/data_normalized.csv", index=False)
data_normalized

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0.547619,1.0,0.715820,1.0,0.000000,0.25,0.2,0.0,0.000000,0.333333,...,0.0,0.000000,0.200,0.000000,0.000000,0.150,0.222222,0.000000,0.294118,1.0
1,0.738095,0.5,0.126700,0.5,0.250000,0.00,0.2,0.0,0.000484,0.666667,...,0.0,0.333333,0.250,0.500000,0.666667,0.250,0.388889,0.066667,0.411765,0.0
2,0.452381,1.0,0.909807,0.5,0.035714,0.25,0.8,0.0,0.001451,1.000000,...,0.0,0.000000,0.175,0.500000,0.666667,0.000,0.000000,0.000000,0.000000,1.0
3,0.357143,0.5,0.923407,0.5,0.071429,0.75,0.2,0.0,0.001935,1.000000,...,0.0,0.000000,0.200,0.500000,0.666667,0.200,0.388889,0.200000,0.000000,0.0
4,0.214286,1.0,0.350036,0.5,0.035714,0.00,0.6,0.0,0.002903,0.000000,...,0.0,0.333333,0.150,0.500000,0.666667,0.050,0.111111,0.133333,0.117647,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,0.428571,0.5,0.559771,0.5,0.785714,0.25,0.6,0.0,0.996613,0.666667,...,0.0,0.333333,0.425,0.500000,0.666667,0.125,0.111111,0.000000,0.176471,0.0
1466,0.500000,1.0,0.365784,0.5,0.178571,0.00,0.6,0.0,0.997097,1.000000,...,0.0,0.333333,0.225,0.833333,0.666667,0.175,0.388889,0.066667,0.411765,0.0
1467,0.214286,1.0,0.037938,0.5,0.107143,0.50,0.2,0.0,0.998065,0.333333,...,0.0,0.333333,0.150,0.000000,0.666667,0.150,0.111111,0.000000,0.176471,0.0
1468,0.738095,0.5,0.659270,1.0,0.035714,0.50,0.6,0.0,0.998549,1.000000,...,0.0,0.000000,0.425,0.500000,0.333333,0.225,0.333333,0.000000,0.470588,0.0


In [91]:
encoder2 = pickle.load(open('./model/encode/BusinessTravel.pkl', 'rb'))
temp = encoder2.transform(['Non-Travel', 'Travel_Frequently', 'Travel_Rarely'])
# print(temp)

scaler.fit(data_encoded['BusinessTravel'].values.reshape(-1,1))   
pickle.dump(scaler, open('./model/scale/BusinessTravel.pkl', 'wb'))
scaler2 = pickle.load(open('./model/scale/BusinessTravel.pkl', 'rb'))


# scaler2 = pickle.load(open('./model/scale/BusinessTravel.pkl', 'rb'))
scaler2.transform(np.array(temp).reshape(-1,1))


array([[0. ],
       [0.5],
       [1. ]])

In [77]:
np.array([1,2]).reshape(-1,1)

array([[1],
       [2]])

In [35]:
X = data_normalized.drop("Attrition", axis=1).values
y = data_normalized["Attrition"].values

In [36]:
cat_index = [data_normalized.columns.get_loc(c) for c in objectList if c in data_normalized.drop('Attrition', axis=1)]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True, stratify=y)
# X_train.to_csv("./data/X_train.csv")

### Seleksi Fitur BGWOPSO

In [38]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

In [14]:
# le = LabelEncoder()
# le.fit_transform([1,2,2,3,3,4,4,5,6])
# le.classes_
# le.fit([9,8,90])
# le.classes_
# le.fit_transform([9,9,9,8,8,90])
# le.classes_
# pickle.dump(le, open('./model/label_encoder.pkl', 'wb'))

array([ 8,  9, 90])

In [ ]:
def fitness(x, X_train, X_test, y_train, y_test):
    alpha = 0.99
    beta = 1-alpha
    if x.ndim == 1:
        x = x.reshape(1, -1)
    loss = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        if np.sum(x[i, :]) > 0:
            model = SVC(C=10, gamma=0.1, kernel='rbf')
            model.fit(X_train[:, x[i,:].astype(bool)], y_train)
            acc = model.score(X_test[:, x[i,:].astype(bool)], y_test)
            error_rate = 1 - acc
            loss[i] = alpha * error_rate + beta * (np.sum(x[i, :]) / X.shape[1])
        else:
            loss[i] = np.inf
    return loss

In [ ]:
lossfunc_bgwopso = functools.partial(fitness, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
optimizer_bgwopso = BGWOPSO(lossfunc_bgwopso, X_train.shape[1], 8, 70)
optimizer_bgwopso.opt()
selected_features_bgwopso = optimizer_bgwopso.gBest_X.astype(bool)
print(selected_features_bgwopso)

In [ ]:
data_selected = data.loc[:, np.append(selected_features_bgwopso, True)]
data_selected.to_csv("./data/data_selected.csv", index=False)

In [ ]:
data_train = pd.DataFrame(X_train[:, selected_features_bgwopso])
data_train[data.columns[-1]] = y_train
data_train.columns = data_selected.columns
data_train.to_csv("./data/data_train.csv", index=False)

data_test = pd.DataFrame(X_test[:, selected_features_bgwopso])
data_test[data.columns[-1]] = y_test
data_test.columns = data_selected.columns
data_test.to_csv("./data/data_test.csv", index=False)

In [ ]:
# print("list of all selected features: ")
for i, feature in enumerate(data.columns[np.where(selected_features_bgwopso)], start=1):
    print(f"{i}. {feature}")

In [ ]:
# 1. BusinessTravel
# 2. DailyRate
# 3. Department
# 4. DistanceFromHome
# 5. EducationField
# 6. EmployeeNumber
# 7. EnvironmentSatisfaction
# 8. Gender
# 9. JobLevel
# 10. JobRole
# 11. JobSatisfaction
# 12. MaritalStatus
# 13. MonthlyIncome
# 14. MonthlyRate
# 15. NumCompaniesWorked
# 16. OverTime
# 17. PercentSalaryHike
# 18. RelationshipSatisfaction
# 19. StockOptionLevel
# 20. TotalWorkingYears
# 21. TrainingTimesLastYear
# 22. YearsInCurrentRole
# 23. YearsSinceLastPromotion
# 24. YearsWithCurrManager

In [ ]:
# import pickle

svm3 = SVC(C=10, gamma=0.1, kernel='rbf')
svm3.fit(X_train[:, selected_features_bgwopso], y_train)
y_pred3 = svm3.predict(X_test[:, selected_features_bgwopso])
# svm3.fit(X_train, y_train)
# y_pred3 = svm3.predict(X_test)
pickle.dump(svm3, open('./model/svm_model.pkl', 'wb'))


print("Accuracy =", round(accuracy_score(y_test,y_pred3)*100, 2))
print("Precision ", round(precision_score(y_test, y_pred3)*100, 2))
print("Recall / Sensifity ", round(recall_score(y_test, y_pred3)*100, 2))
print("F1 ", round(f1_score(y_test, y_pred3)*100, 2))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred3).ravel()
specificity = tn / (tn+fp)
print("Specificity ", round(specificity*100, 2))
print("TN = ", tn)
print("FP = ", fp)
print("FN = ", fn)
print("TP = ", tp)

In [ ]:
# evaluasi_test_data = data_test.copy()
# evaluasi_test_data["Predicted"] = y_pred3
# evaluasi_test_data.to_csv("./data/evaluasi_test_data.csv", index=False)

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
# confusion_matrix = metrics.confusion_matrix(y_test, y_pred3)
# cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
# cm_display.plot()
# plt.savefig("./images/confusion_matrix_employee.jpg")

In [ ]:
# predicted = ["predicted_false", "predicted_true"]
# actual = ["actual_false", "actual_true"]
# df_cf = pd.DataFrame(columns = predicted, index = actual)
# df_cf["predicted_false"] = [tn, fn]
# df_cf["predicted_true"] = [fp, tp]

In [ ]:
# C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
# g = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
C = [1, 10, 100, 1000]
g = [0.001, 0.01, 0.1, 1]
df_acc = pd.DataFrame(columns = C, index = g)
df_prec = pd.DataFrame(columns = C, index = g)
df_sens = pd.DataFrame(columns = C, index = g)
df_f1 = pd.DataFrame(columns = C, index = g)
df_spec = pd.DataFrame(columns = C, index = g)

for i in C:
    for j in g:
        # print(i, j)
        svm3 = SVC(C=i, gamma=j, kernel='rbf')
        # svm3 = SVC(C=i, gamma=j, kernel='poly')
        # svm3 = SVC(C=i, gamma=j, kernel='sigmoid')
        svm3.fit(X_train[:, selected_features_bgwopso], y_train)
        y_pred3 = svm3.predict(X_test[:, selected_features_bgwopso])
        # svm3.fit(X_train, y_train)
        # y_pred3 = svm3.predict(X_test)

        accuracy = round(accuracy_score(y_test,y_pred3)*100, 2)
        df_acc[i][j] = accuracy
        
        precision = round(precision_score(y_test, y_pred3)*100, 2)
        df_prec[i][j] = precision

        sensifity = round(recall_score(y_test, y_pred3)*100, 2)
        df_sens[i][j] = sensifity

        F1 = round(f1_score(y_test, y_pred3)*100, 2)
        df_f1[i][j] = F1

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred3).ravel()
        specificity = round(tn/(tn+fp)*100, 2)
        df_spec[i][j] = specificity

print("Accuracy")
print(df_acc)
df_acc.to_excel("cm/accuracy.xlsx")

print("Precision")
print(df_prec)
df_prec.to_excel("cm/precision.xlsx")

print("Sensifity")
print(df_sens)
df_sens.to_excel("cm/sensifity.xlsx")

print("F1 Score")
print(df_f1)
df_f1.to_excel("cm/f1.xlsx")

print("Specifity") 
print(df_spec) 
df_spec.to_excel("cm/specifity.xlsx")


In [ ]:
from sklearn.svm import LinearSVC

matrix_list = ["Accuracy", "Precision", "Sensifity", "F1", "Specificity"]
df_svm_linear = pd.DataFrame(columns = C, index = matrix_list)

for i in C:
    svm3 = LinearSVC(C=i)
    # svm3 = SVC(C=i, gamma=j, kernel='linear')
    svm3.fit(X_train[:, selected_features_bgwopso], y_train)
    y_pred3 = svm3.predict(X_test[:, selected_features_bgwopso])
    # svm3.fit(X_train, y_train)
    # y_pred3 = svm3.predict(X_test)

    accuracy = round(accuracy_score(y_test,y_pred3)*100, 2)
    df_svm_linear[i]["Accuracy"] = accuracy
    
    precision = round(precision_score(y_test, y_pred3)*100, 2)
    df_svm_linear[i]["Precision"] = precision

    sensifity = round(recall_score(y_test, y_pred3)*100, 2)
    df_svm_linear[i]["Sensifity"] = sensifity

    F1 = round(f1_score(y_test, y_pred3)*100, 2)
    df_svm_linear[i]["F1"] = F1

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred3).ravel()
    specificity = round(tn/(tn+fp)*100, 2)
    df_svm_linear[i]["Specificity"] = specificity

df_svm_linear.to_excel("cm/matrix_svm_linear.xlsx")
print(df_svm_linear)

In [ ]:
evaluasi_test_data = pd.DataFrame(X_test[:, selected_features_bgwopso])
evaluasi_test_data.columns = data.columns[np.where(selected_features_bgwopso)]
evaluasi_test_data["Actual"] = y_test
evaluasi_test_data["Predicted"] = y_pred3
# evaluasi_test_data.to_excel("evaluasi_test_data.xlsx")

### Seleksi Fitur GWO

In [ ]:
# lossfunc_gwo = functools.partial(fitness, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
# fit_optimizer_gwo = GWO(lossfunc_gwo, 0, 1, X_train.shape[1], 10, 20)
# selected_features_gwo = np.where(fit_optimizer_gwo>0.5)[0]
# print(selected_features_gwo)

In [ ]:
# svm2 = SVC(C=10, gamma=0.1, kernel='rbf')
# svm2.fit(X_train[:, selected_features_gwo], y_train)
# y_pred2 = svm2.predict(X_test[:, selected_features_gwo])

# print("Accuracy ", accuracy_score(y_test,y_pred2))
# print("Precision ", precision_score(y_test, y_pred2))
# print("Recall / Sensifity ", recall_score(y_test, y_pred2))
# print("F1 ", f1_score(y_test, y_pred2))
# tn, fp, fn, tp = confusion_matrix(y_test, y_pred2).ravel()
# specificity = tn / (tn+fp)
# print("Specificity ", specificity)

In [ ]:
# import sys
# import numpy
# numpy.set_printoptions(threshold=sys.maxsize)
# print(X_test[:, selected_features_gwo])

### Tanpa Seleksi Fitur

In [ ]:
# svm1 = SVC(C=10, gamma=0.1, kernel='rbf')
# svm1.fit(X_train, y_train).score(X_test, y_test)
# y_pred1 = svm1.predict(X_test)

# print("Accuracy ", accuracy_score(y_test,y_pred1))
# print("Precision ", precision_score(y_test, y_pred1))
# print("Recall / Sensifity ", recall_score(y_test, y_pred1))
# print("F1 ", f1_score(y_test, y_pred1))
# tn, fp, fn, tp = confusion_matrix(y_test, y_pred1).ravel()
# specificity = tn / (tn+fp)
# print("Specificity ", specificity)

In [ ]:
# print("list of all selected features: ")
# for i, feature in enumerate(data.columns[np.where(selected_features_gwo)], start=1):
#     print(f"{i}. {feature}")

### PSO

In [ ]:

# from Fitness import Data
# import PSO

# d = Data('employee_class_on_right.csv',False, 3) # Object for Data
# dim = d.getDimension()  # Dimensionality of the Features
# selected_pso = PSO.run(d,dim,3) # invoking
# selected_pso_bool = selected_pso.astype(bool)
# print(selected_pso_bool)

In [ ]:
# svm4 = SVC(C=10, gamma=0.1, kernel='rbf')
# svm4.fit(X_train[:, selected_pso_bool], y_train)
# y_pred4 = svm4.predict(X_test[:, selected_pso_bool])

# print("Accuracy ", accuracy_score(y_test,y_pred4))
# print("Precision ", precision_score(y_test, y_pred4))
# print("Recall / Sensifity ", recall_score(y_test, y_pred4))
# print("F1 ", f1_score(y_test, y_pred4))
# tn, fp, fn, tp = confusion_matrix(y_test, y_pred4).ravel()
# specificity = tn / (tn+fp)
# print("Specificity ", specificity)